<a href="https://colab.research.google.com/github/Caue-Aramaki/random_pushes_caue_aramaki/blob/main/command_parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Parsing commans lines and returning the computed values**

In [ ]:
def convert_num_from_x_to_y(num, base_x=10, base_y=2):
  try:
    import math
    import numpy as np

    base_x = int(base_x)
    base_y = int(base_y)

    x_bases = str(num)
    decimal_value = 0
    x_len = len(x_bases)

    for digit_index in range(0, x_len):
      decimal_value += int(x_bases[digit_index]) * (base_x ** (x_len - digit_index - 1))
    
    value = decimal_value
    converted = ""

    exp = int(np.round(math.log(value, base_y)))

    for exp_index in range(0, exp + 1):

      temp = base_y ** (exp - exp_index)
      
      for coef_index in range(0, base_y + 1):

        if (temp * coef_index) > value:
          value = value - temp * (coef_index - 1)
          converted += str(coef_index - 1)
          break
    return int(converted)
  except Exception as problem:
    return f"Error: {str(problem)}"

In [ ]:
# we want to create nested functions within a line.
# let us define some functions:
def multiply_three(input_1=None, input_2=None, input_3=None, input_list=None):
  try:
    if input_1 != None:
      return int(input_1) * int(input_2) * int(input_3)
    else:
      return int(input_list[0]) * int(input_list[1]) * int(input_list[2])
  except Exception as problem:
    return f"in multiply_three(): {str(problem)}"

def add_one(input=None, input_list=None):
  try:
    if input != None:
      return int(input) + 1
    else:  
      return int(input_list[0]) + 1
  except Exception as problem:
    return f"in add_one(): {str(problem)}"

def linear(input=None, input_list=None):
  try:
    if input != None:
      return int(input)
    else:  
      return int(input_list[0])
  except Exception as problem:
    return f"in add_one(): {str(problem)}"

# let us create a prototype sentence
sentence = "1, 2, multiply_three(1, 2, multiply_three(2, 7, 1)), add_one(1)"

# the functions work. notice that comma split gives us the number of total parameters of our nested commands
len(sentence.split(',')), multiply_three(1, 2, 3), add_one("a")

(8, 6, "in add_one(): invalid literal for int() with base 10: 'a'")

In [ ]:
def parse_functions(input_string):
  try:
    if input_string.find(",") == -1 and input_string.find("(") == -1:
      return [input_string]
    else:
      # We need to separate the commas from 1, 3, add(1, 3), 2, add(3), but no the ones inside parenthesis!!
      new_string = input_string.replace(" ", "") #get rid of spaces
      temp_string = ""
      parenthesis_counter = 0

      for index in range(len(new_string)): # lets get rid of the commas outside the func
        if new_string[index] == "(":
          parenthesis_counter += 1
        if new_string[index] == ")":
          parenthesis_counter -= 1
        
        if parenthesis_counter == 0 and new_string[index] == ",": # if its a comma outside the func, replace it with ;
          temp_string += ";"
        else:
          temp_string += new_string[index]
      # end
      temp_list =  temp_string.split(";")

      new_list = []
      new_string = ""

      for string in temp_list: # for every string in the new list, lets extract the parameters on each of them

        if string.find("(") == -1:
          output_list = string
          new_list.append(output_list)
        else:
          output_list = []
          count = 0
          temp = ""
          detect_function = ""
          temp_function = ""

          for character_index in range(0, len(string)): # for each string lets iterate and extract features
            if count == 0:
              detect_function += string[character_index]
            if string[character_index] == ")":
              count -= 1
            if count > 0:
              temp += string[character_index]
            else:
              if len(temp) > 0:
                output_list.append([temp_function.replace("(", ""), temp]) # add the params to the list
                temp = ""
                temp_function = ""
            if string[character_index] == "(":
              count += 1
            if count == 1:
              temp_function += detect_function
              detect_function = ""
          # end of loop
          new_list.append(output_list[0])
      # end of loop

      output_list = new_list

      for index in range(0, len(new_list)):
        if type(new_list[index]) == list:
          output_list[index][1] = parse_functions(new_list[index][1])

      return output_list

  except Exception as problem:
    return problem

In [ ]:
print(parse_functions(sentence.replace("$ ", "")))

['1', '2', ['multiply_three', ['1', '2', ['multiply_three', ['2', '7', '1']]]], ['add_one', ['1']]]


In [ ]:
# that we have a parsed tree of functions, we need to output a list of results.

# [1, 2, multiply_three(1, 2, add_one(2))] needs to be the list [1, 2, 6]

parsed_tree = parse_functions(sentence.replace("$ ", "")) # generate our test tree

# for this, we need to create a key list. for each string function,
# we will have the respective function object.

key_list = [
            ["multiply_three", multiply_three],
            ["add_one", add_one],
            ["linear", linear]
]

def prototype_find_command(input_command, parameter_list):
  try:
    found = False

    for bucket in key_list:
      if bucket[0] == input_command:
        found = bucket[1]
    
    return found(input_list=parameter_list)
  
  except Exception as problem:
    return "Error in find_command: " + str(problem)

# with this function, we can do find_command(command, parameter_list)
prototype_find_command("add_one", [1]), prototype_find_command("multiply_three", [1, 2, 3])
# this allows for name changing without much complications,
# while also limiting the user access to script only functions.

# it can be used normally with user friendly parameters, but also
# provide the list flexibility for our current usage.

(2, 6)

In [ ]:
# now, on to our recursion tree!
# recall that each node of our tree is on the following format:

# node = [scalars, ["function", parameter_list]]

# what we currently need to do is: node = [scalars, find_command("function", parameter_list)].
# only the above statement is not enough, because we do not call find_command
# enough times for a recursion. we need to check if there are commands inside 
# the parameter list, so that we can call them.

#

def find_command(input_command, parameter_list):
  try:
    found = False

    for bucket in key_list:
      if bucket[0] == input_command:
        found = bucket[1]
    # we found our command, let us check if the parameter list has functions inside them.

    scalar_list = [] # we will use this as parameters instead, and pass values to it.

    for element in parameter_list:
      if type(element) == list:
        scalar_list.append(find_command(element[0], element[1])) # this is where our recursion is
      else:
        scalar_list.append(element)

    return found(input_list=scalar_list)
  
  except Exception as problem:
    return "Error in find_command: " + str(problem)
# end

# works fine with no functions inside them
print("Non nested:")
print(find_command("add_one", [1]), find_command("multiply_three", [1, 2, 3]))

# Works fine with nested commands as well!!
print("Nested:")
print(find_command("multiply_three", [1, 2, ["add_one", [2]]]))
print(find_command("add_one", [["add_one", [1]]]))

Non nested:
2 6
Nested:
6
3


In [ ]:
# remember our parsing takes in a string and returns a list, which is a node.
# hence, we can not do find_command(parse_functions(input))

# this is our list. we a also want a list as output.
print(parsed_tree)

# we will use the linear() command to assist us

result_list = []

for item in parsed_tree:
  if type(item) == list:
    result_list.append(find_command("linear", [item]))
  else:
    result_list.append(int(item))

# the magic
print(result_list)

['1', '2', ['multiply_three', ['1', '2', ['multiply_three', ['2', '7', '1']]]], ['add_one', ['1']]]
[1, 2, 28, 2]


In [ ]:
def recurse_evalue(command_line):
  parsed_tree = parse_functions(str(command_line))

  result_list = []
  for item in parsed_tree:
    if type(item) == list:
      result_list.append(find_command("linear", [item]))
    else:
      result_list.append(int(item))
  return result_list

recurse_evalue("add_one(1)")

[2]